# 공공데이터포털 - 한국 천문원 데이터
출처 https://astro.kasi.re.kr/life/post/calendarData

Non Data Leakage 데이터로 특일정보와 해달출몰시간예측 값이다. 2018년12월31일 기준 3개월 이상 데이터에 접근이 가능하고, 2021년 12월 시점에도 2022년 상하반기 데이터에 적근이 가능한 것으로 확인되었다.

해당 데이터는 옛 조상들이 날씨를 예측할 때 사용했던 값이고, 날씨와 매우 상관관계가 높은 데이터들이다

In [ ]:
### 뼈대가 될 train DATA LOADING
import pandas as pd
import numpy as np
from scipy import stats
import os
import io

PATH = "/content/drive/MyDrive/dacon/gas/data"

TRAIN_SOURCE = os.path.join(PATH, "한국가스공사_시간별 공급량_20181231.csv")
TEST_SOURCE = os.path.join(PATH, "test.csv")

df_train = pd.read_csv(TRAIN_SOURCE, encoding = "CP949", engine='python')
df_test = pd.read_csv(TEST_SOURCE)

df_train.head()

d_map = {}
for i, d in enumerate(df_train['구분'].unique()):
    d_map[d] = i

df_train['구분'] = df_train['구분'].map(d_map)

df_train['연월일'] = pd.to_datetime(df_train['연월일'])
df_train['year'] = df_train['연월일'].dt.year
df_train['month'] = df_train['연월일'].dt.month
df_train['day'] = df_train['연월일'].dt.day
df_train['weekday'] = df_train['연월일'].dt.weekday

df_train.head(5)

,연월일,시간,구분,공급량,year,month,day,weekday
0,2013-01-01,1,0,2497.129,2013,1,1,1
1,2013-01-01,2,0,2363.265,2013,1,1,1
2,2013-01-01,3,0,2258.505,2013,1,1,1
3,2013-01-01,4,0,2243.969,2013,1,1,1
4,2013-01-01,5,0,2344.105,2013,1,1,1


In [ ]:
pip install xmltodict

In [ ]:
import pandas as pd
import requests
import os
import json
import xmltodict

Encoding = ###Encoding###
Decoding = ###Decoding###

days_info = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService"
days_columns = ['getHoliDeInfo','getRestDeInfo','getAnniversaryInfo','get24DivisionsInfo','getSundryDayInfo']
days_params ={'serviceKey' : Decoding, 'pageNo' : '1', 'numOfRows' : '10', 'solYear' : '2019', 'solMonth' : '02'}

sunrise_info = "http://apis.data.go.kr/B090041/openapi/service/RiseSetInfoService"
sunrise_columns = ['getLCRiseSetInfo']
sunrise_ ={'serviceKey' : Decoding, 'locdate' : '20150101', 'longitude' : 127.84819444, 'latitude' : 36.45166667, 'dnYn' : 'Y'}

url = os.path.join(days_info, days_columns[1])


In [ ]:
def data_call(url, params):
  response = requests.get(url, params=params)

  xml_parse = xmltodict.parse(response.content)     # string인 xml 파싱
  xml_dict = json.loads(json.dumps(xml_parse))
  try:
    df = pd.json_normalize(xml_dict['response']['body']['items']['item']) #Results contain the required data
  except TypeError:
    df = pd.DataFrame()
  return df

In [ ]:
## 특일정보.csv 다운 및 처리 
## 특일정보는 몇년 후 데이터도 접근이 가능하기때문에 2018-12-31일당시에 2019-03-31데이터 접근가능

In [ ]:
df = pd.DataFrame()
for col in days_columns:
  for yr in range(2013, 2020):
    for mth in range(1,13):
      url = os.path.join(days_info, col)
      days_params ={'serviceKey' : Decoding, 'pageNo' : '1', 'numOfRows' : '10', 'solYear' : f"{yr}", 'solMonth' : f"{mth:02}"}
      df = pd.concat([df, data_call(url, days_params)])
df

In [ ]:
PATH = "/content/drive/MyDrive/dacon/gas/data"
df = df.drop(columns = ['Unnamed: 0', 'dateKind', 'remarks', 'kst', 'seq'])
df = df.rename(columns = {'dateName':'명칭', 'isHoliday':'공공기관 휴일여부', 'sunLongitude':'태양황경', 'locdate':'일자'})
df = df[['일자'] + [col for col in df.columns if col != '일자']]
df.to_csv(os.path.join(PATH, "특일정보.csv"), index=False)
df


In [ ]:
import pandas as pd
tmp = pd.read_csv('/content/특일정보.csv')
tmp1 = tmp[tmp['isHoliday']=='Y']
tmp1.columns = ['일자', '공휴일', 'isHoliday', '태양황경']
tmp2 = tmp[tmp['공공기관 휴일여부']=='N']
tmp2.columns = ['일자', '비공휴일', '공공기관 휴일여부', '태양황경']
tmp = pd.merge(left = tmp1, right=tmp2, how='left').reset_index(drop=True)
tmp = tmp.drop(columns = '공공기관 휴일여부')

tfj = ['입춘', '경칩', '청명', '입하', '망종', '소서', '입추', '백로', '한로', '입동', '대설', '소한', '우수', '춘분', '곡우', '소만', '하지', '대서', '처서', '추분', '상강', '소설', '동지', '대한', '초복', '중복', '말복']
tmp['24절기 및 삼복'] = tmp['비공휴일'].apply(lambda x: x if x in tfj else np.nan)
tmp = pd.merge(left = tmp[tmp['24절기 및 삼복'].isnull()].drop(columns = '24절기 및 삼복'), right=tmp[tmp['24절기 및 삼복'].notnull()][['일자', '24절기 및 삼복']], how='left').reset_index(drop=True)
tmp = tmp.drop_duplicates(keep='first')

#20171003에는 개천절과 추석이 동시에 있어 개천절은 드롭
tmp = tmp.drop(index = tmp[(tmp['공휴일']=='개천절') & (tmp['일자']==20171003)].index)
tmp

In [ ]:
import pandas as pd
import numpy as np
tmp = pd.read_csv('/content/특일정보.csv')
#Datekind {1:"공휴일", 2:"비공휴일", 3:"24절기삼복", 4:"태양황경", 5:"삼복잡절"}
tmp.loc[tmp.dateName.isin(["초복", "중복", "말복", "한식", "단오"]), "dateKind"] = 5
tmp['isHoliday'] = tmp['isHoliday'].map({"Y":1, "N":0})
tmp['24절기삼복'] = tmp['dateName']
tmp = tmp.rename(columns={'locdate' : '일자'})
tmp['24절기삼복'] = tmp['24절기삼복'].map({
"입춘" : 1, "우수" : 2, "경칩" : 3, "춘분" : 4, "청명" : 5, "곡우" : 6,
"입하" : 7, "소만" : 8, "망종" : 9, "하지" : 10, "소서" : 11, "대서" : 12,
"입추" : 13, "처서" : 14, "백로" : 15, "추분" : 16, "한로" : 17, "상강" : 18,
"입동" : 19, "소설" : 20, "대설" : 21, "동지" : 22, "소한" : 23, "대한" : 24,
})
tmp = tmp[tmp['일자']<20190401]
tmp = tmp.drop_duplicates()
tmp = tmp.sort_values(by='isHoliday').drop_duplicates(subset='일자', keep="last")
tmp['일자'].value_counts()

tmp = tmp.drop(['Unnamed: 0', "seq", "remarks", "kst", "sunLongitude", "dateKind", "dateName"], axis=1)

tmp.to_csv("/content/drive/MyDrive/dacon/gas/data/non_leak/특일정보.csv", index=False)
tmp

,isHoliday,일자,24절기삼복
414,0,20170403,NaN
559,0,20130923,16.0
558,0,20130907,15.0
557,0,20130823,14.0
556,0,20130807,13.0
...,...,...,...
80,1,20170815,NaN
81,1,20171002,NaN
82,1,20171003,NaN
70,1,20170127,NaN


In [ ]:
### 해달출몰시간 파일 다운 및 처리
## 해달출몰시간의 데이터는 2018-12-31일당시에 2019-03-31데이터 접근가능.
## 천문연구원에서 직접 예측을 하고 그 데이터를 공공포털에 넣어두거나 직접 사이트에 업데이트를 하는데 3개월 이상의 미래 데이터 접근 가능 확인

In [ ]:
dff = pd.DataFrame()
for col in sunrise_columns:
  for locdate in pd.date_range(start="2013-01-01",end="2019-12-31").strftime("%Y%m%d").tolist():
      url = os.path.join(sunrise_info, col)
      params ={'serviceKey' : Decoding, 'locdate' : f'{locdate}', 'longitude' : 127.84819444, 'latitude' : 36.45166667, 'dnYn' : 'Y'}
      dff = pd.concat([dff, data_call(url, params)])
dff

In [ ]:
PATH = "/content/drive/MyDrive/dacon/gas/data"
dff.to_csv(os.path.join(PATH, "해달출몰시간.csv"))

In [ ]:
for col in sunrise_columns:
  for locdate in pd.date_range(start="2017-03-06",end="2018-13-31").strftime("%Y%m%d").tolist():
      url = os.path.join(sunrise_info, col)
      params ={'serviceKey' : Decoding, 'locdate' : f'{locdate}', 'longitude' : 127.84819444, 'latitude' : 36.45166667, 'dnYn' : 'Y'}
      dff = pd.concat([dff, data_call(url, params)])
dff

,aste,astm,civile,civilm,latitude,latitudeNum,location,locdate,longitude,longitudeNum,moonrise,moonset,moontransit,naute,nautm,sunrise,sunset,suntransit
0,1855,0607,1751,0710,3624,36.4000000,상주,20130101,12809,128.1500000,2110,0939,025905,1824,0638,0739,1723,123050
0,1856,0607,1752,0711,3624,36.4000000,상주,20130102,12809,128.1500000,2210,1011,034428,1824,0638,0739,1724,123118
0,1856,0607,1753,0711,3624,36.4000000,상주,20130103,12809,128.1500000,2310,1043,042952,1825,0639,0739,1724,123145
0,1857,0607,1754,0711,3624,36.4000000,상주,20130104,12809,128.1500000,----,1115,051604,1826,0639,0739,1725,123213
0,1858,0608,1755,0711,3624,36.4000000,상주,20130105,12809,128.1500000,0013,1149,060358,1827,0639,0739,1726,123240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2013,0454,1911,0555,3624,36.4000000,상주,20190328,12809,128.1500000,0121,1125,062300,1942,0524,0620,1845,123238
0,2014,0452,1912,0553,3624,36.4000000,상주,20190329,12809,128.1500000,0213,1215,071400,1943,0523,0619,1846,123220
0,2015,0450,1913,0552,3624,36.4000000,상주,20190330,12809,128.1500000,0300,1308,080300,1943,0521,0618,1847,123202
0,2016,0449,1914,0550,3624,36.4000000,상주,20190331,12809,128.1500000,0341,1402,085000,1944,0520,0616,1848,123144


In [ ]:
dff.sort_values(by="locdate")

,aste,astm,civile,civilm,latitude,latitudeNum,location,locdate,longitude,longitudeNum,moonrise,moonset,moontransit,naute,nautm,sunrise,sunset,suntransit
0,1855,0607,1751,0710,3624,36.4000000,상주,20130101,12809,128.1500000,2110,0939,025905,1824,0638,0739,1723,123050
0,1856,0607,1752,0711,3624,36.4000000,상주,20130102,12809,128.1500000,2210,1011,034428,1824,0638,0739,1724,123118
0,1856,0607,1753,0711,3624,36.4000000,상주,20130103,12809,128.1500000,2310,1043,042952,1825,0639,0739,1724,123145
0,1857,0607,1754,0711,3624,36.4000000,상주,20130104,12809,128.1500000,----,1115,051604,1826,0639,0739,1725,123213
0,1858,0608,1755,0711,3624,36.4000000,상주,20130105,12809,128.1500000,0013,1149,060358,1827,0639,0739,1726,123240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1947,0533,1848,0632,3624,36.4000000,상주,20170301,12809,128.1500000,0834,2113,144950,1917,0603,0658,1822,123940
0,1948,0531,1849,0631,3624,36.4000000,상주,20170302,12809,128.1500000,0911,2220,154157,1918,0601,0657,1823,123929
0,1949,0530,1849,0630,3624,36.4000000,상주,20170303,12809,128.1500000,0951,2327,163522,1919,0600,0656,1824,123916
0,1950,0529,1850,0628,3624,36.4000000,상주,20170304,12809,128.1500000,1033,----,173017,1920,0559,0654,1825,123903


In [ ]:
df = pd.read_csv(os.path.join(PATH, "해달출몰시간.csv"))
df = df.drop(columns = ['Unnamed: 0', 'latitude', 'latitudeNum', 'location', 'longitude', 'longitudeNum'])
df = df.rename(columns = {'locdate':'일자', 'sunrise':'일출', 'suntransit':'일중', 'sunset':'일몰', 'moonrise':'월출', 
                          'moontransit':'월중', 'moonset':'월몰', 'civilm':'시민박명(아침)', 'civile':'시민박명(저녁)', 
                          'nautm':'항해박명(아침)', 'naute':'항해박명(저녁)', 'astm':'천문박명(아침)', 'aste':'천문박명(저녁)'})
df = df[['일자'] + [col for col in df.columns if col != '일자']]
df

In [ ]:
dd = pd.read_csv(os.path.join(PATH, "해달출몰시간.csv"))
dd['월출'] = dd['월출'].replace(['----'], method='bfill')
dd['월중'] = dd['월중'].replace(['------'], method='bfill')
dd['월몰'] = dd['월몰'].replace(['----'], method='bfill')

f = pd.DataFrame()
f['일자'] = dd['일자']
f['천문박명 시(저녁)'] = dd['천문박명(저녁)'].astype(str).str.zfill(4).str.slice(0,2).astype(int)
f['천문박명 분(저녁)'] = dd['천문박명(저녁)'].astype(str).str.zfill(4).str.slice(2,4).astype(int)
f['천문박명 시(아침)'] = dd['천문박명(아침)'].astype(str).str.zfill(4).str.slice(0,2).astype(int)
f['천문박명 분(아침)'] = dd['천문박명(아침)'].astype(str).str.zfill(4).str.slice(2,4).astype(int)
f['시민박명 시(저녁)'] = dd['시민박명(저녁)'].astype(str).str.zfill(4).str.slice(0,2).astype(int)
f['시민박명 분(저녁)'] = dd['시민박명(저녁)'].astype(str).str.zfill(4).str.slice(2,4).astype(int)
f['시민박명 시(아침)'] = dd['시민박명(아침)'].astype(str).str.zfill(4).str.slice(0,2).astype(int)
f['시민박명 분(아침)'] = dd['시민박명(아침)'].astype(str).str.zfill(4).str.slice(2,4).astype(int)
f['항해박명 시(저녁)'] = dd['항해박명(저녁)'].astype(str).str.zfill(4).str.slice(0,2).astype(int)
f['항해박명 분(저녁)'] = dd['항해박명(저녁)'].astype(str).str.zfill(4).str.slice(2,4).astype(int)
f['항해박명 시(아침)'] = dd['항해박명(아침)'].astype(str).str.zfill(4).str.slice(0,2).astype(int)
f['항해박명 분(아침)'] = dd['항해박명(아침)'].astype(str).str.zfill(4).str.slice(2,4).astype(int)
f['월출 시'] = dd['월출'].astype(str).str.zfill(4).str.slice(0,2).astype(int)
f['월출 분'] = dd['월출'].astype(str).str.zfill(4).str.slice(2,4).astype(int)
f['월중 시'] = dd['월중'].astype(str).str.zfill(6).str.slice(0,2).astype(int)
f['월중 분'] = dd['월중'].astype(str).str.zfill(6).str.slice(2,4).astype(int)
f['월몰 시'] = dd['월몰'].astype(str).str.zfill(4).str.slice(0,2).astype(int)
f['월몰 분'] = dd['월몰'].astype(str).str.zfill(4).str.slice(2,4).astype(int)
f['일출 시'] = dd['일출'].astype(str).str.zfill(4).str.slice(0,2).astype(int)
f['일출 분'] = dd['일출'].astype(str).str.zfill(4).str.slice(2,4).astype(int)
f['일몰 시'] = dd['일몰'].astype(str).str.zfill(4).str.slice(0,2).astype(int)
f['일몰 분'] = dd['일몰'].astype(str).str.zfill(4).str.slice(2,4).astype(int)
f

In [ ]:
url = os.path.join(sunrise_info, col)
params ={'serviceKey' : Decoding, 'locdate' : '20170307', 'longitude' : 127.84819444, 'latitude' : 36.45166667, 'dnYn' : 'Y'}
response = requests.get(url, params=params)

xml_parse = xmltodict.parse(response.content)     # string인 xml 파싱
xml_dict = json.loads(json.dumps(xml_parse))
xml_dict

{'response': {'body': {'items': {'item': {'aste': '1953',
     'astm': '0525',
     'civile': '1853',
     'civilm': '0624',
     'latitude': '3624',
     'latitudeNum': '36.4000000',
     'location': '상주',
     'locdate': '20170307',
     'longitude': '12809',
     'longitudeNum': '128.1500000',
     'moonrise': '1307',
     'moonset': '0238',
     'moontransit': '202103',
     'naute': '1923',
     'nautm': '0555',
     'sunrise': '0650',
     'sunset': '1827',
     'suntransit': '123822'}},
   'numOfRows': '10',
   'pageNo': '1',
   'totalCount': '1'},
  'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}}}

# 날씨데이터

아래 파일들은 - https://data.kma.go.kr/climate/windChill/selectWindChillChart.do?pgmNo=111 에서 직접 받아서 수작업 한 파일들임

아래에 수작업 예시 코드가 있다. 2018년까지 데이터를 받은 후 2019년 row를 만든다음 빈칸에 0을 넣어 저장한 파일들.

데이터 채워주는 건 추후에 pseudo labeling형식으로 prophet 예측값들로 채워줌

In [ ]:
"""
장마_평균장마기간_2013_2018.csv
기온분석_평균기온_2013_2018.csv
조건별통계_평균기온_2013_2018.csv
조건별통계_평균습도_2013_2018.csv
조건별통계_평균풍속_2013_2018.csv
조건별통계_평균강수량_2013_2018.csv
강수량분석_강수량_2013_2018.csv
기상현상일수_강수일수_2013_2018.csv
기상현상일수_폭염일수_그래프_2013_2018.csv
기상현상일수_폭염일수_2013_2018.csv
기상현상일수_폭풍일수_2013_2018.csv
응용기상분석_열지수_서울_2013_2018.csv
응용기상분석_체감온도_서울_2013_2018.csv
응용기상분석_냉난방도일_2013_2018.csv
"""

### 예시 전처리 코드

In [ ]:
df_monthly = pd.DataFrame()
df_monthly['일자'] = pd.read_csv(os.path.join(PATH, "기온분석_평균기온_2013_2018.csv")).sort_values(by='일자').reset_index(drop='True')['일자']
df_monthly

df = pd.read_csv(os.path.join(PATH, "기상현상일수_폭풍일수_2013_2018.csv"))
df.stack().to_frame().T.values.tolist()
l = df.stack().to_frame().T.values.tolist()
l = l[0]
l = [x for x in l if x < 2000]
v = [l[x:x+12] for x in range(0, len(l), 14)]
s = [l[x+12] for x in range(0, len(l), 14)]
r = [l[x+13] for x in range(0, len(l), 14)]
s = [[x]*12 for x in s]
s = [item for sublist in s for item in sublist]
r = [[x]*12 for x in r]
r = [item for sublist in r for item in sublist]
v  = [item for sublist in v for item in sublist]

df_monthly['폭풍일수'] = v
df_monthly['폭풍일수_연합계'] = s
df_monthly['폭풍일수_순위'] = r

df_monthly.to_csv(os.path.join(PATH, "기상현상일수_폭풍일수_2013_2018.csv"), index=False)
df = pd.read_csv(os.path.join(PATH, "기상현상일수_폭풍일수_2013_2018.csv"))
df
